In [21]:
import json
import pandas as pd
import numpy as np

# Lab 5

### Preparation
Download the sample data file fullweeklycoviddatajson.txt (its in Lab 6) from MyCourses. Upload the file to your JupyterLab folder

Lab 5
### **The objective of this lab is to produce a mapper**
which generates data for a reducer which calculates the number of countries during week 2020-52 with a COVID rate in the following 5 internal [0-1],[1-5],[5-10],[10,50] and [50,infinity) 

1. Take the Mapper Example (its in Lab 6) and create a Python3 notebook using that same code

2. Replace the input file with fullweeklycoviddatajson.txt

3. Replace the output file with lab5mapperout.txt

4. In the keyvaluemapper function,calculate a key depending on the value of the field <notification_rate_per_100000_population_14-days>:

    
    - if data["notification_rate_per_100000_population_14-days"] <1 then key="INT0-1" 

    - else if  data["notification_rate_per_100000_population_14-days"] <5 then key="INT1-5"

    - etc....

    - set the value to just {"count":1} as this record just represents one country, and the reducer just needs to calculate the number of countries

    - call the mapperoutput only if data["year_week"]=="2020-52"

5. Execute

6. Verify the result in lab5mapperout.txt

In [28]:
# Exploring input data
inputs = []
with open('fullweeklycoviddatajson.txt') as f:
    for i,line in enumerate(f):
        row_data = json.loads(line)
        inputs.append(row_data)

print(inputs[0:2])
input_df = pd.DataFrame(inputs)
input_df.tail()

[{'dateRep': '28/12/2020', 'year_week': '2020-52', 'cases_weekly': 1994, 'deaths_weekly': 88, 'countriesAndTerritories': 'Afghanistan', 'geoId': 'AF', 'countryterritoryCode': 'AFG', 'popData2019': 38041757.0, 'continentExp': 'Asia', 'notification_rate_per_100000_population_14-days': 7.19}, {'dateRep': '21/12/2020', 'year_week': '2020-51', 'cases_weekly': 740, 'deaths_weekly': 111, 'countriesAndTerritories': 'Afghanistan', 'geoId': 'AF', 'countryterritoryCode': 'AFG', 'popData2019': 38041757.0, 'continentExp': 'Asia', 'notification_rate_per_100000_population_14-days': 6.56}]


,dateRep,year_week,cases_weekly,deaths_weekly,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,notification_rate_per_100000_population_14-days
9359,20/04/2020,2020-16,11,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,0.11
9360,13/04/2020,2020-15,5,2,Zimbabwe,ZW,ZWE,14645473.0,Africa,0.05
9361,06/04/2020,2020-14,2,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,0.05
9362,30/03/2020,2020-13,5,1,Zimbabwe,ZW,ZWE,14645473.0,Africa,0.05
9363,23/03/2020,2020-12,2,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,NaN


In [32]:
# read data from txt file
finput=open('fullweeklycoviddatajson.txt','r')
# output sorted result from mapper to the txt file as below
foutput=open('lab5mapperout.txt','w')

## Mapper Output Function

In [33]:
def mapperoutput(key,value):
    dataline=key+"\t"+json.dumps(value)+"\n"
    foutput.write(dataline)

## KeyValue Mapper function

In [34]:
# call this function for each JSON line
def keyvaluemapper(data):
    #print(data)
    i = data["notification_rate_per_100000_population_14-days"]
    # because the key column contains some NaN, here I assume skip NaN
    if i is None:  
        key = "INT-NaN"
    # for interval [0-1],[1-5],[5-10],[10,50] and [50,infinity)
    elif i <1:
        key = "INT0-1"
    elif i <5:
        key = "INT1-5"
    elif i <10:
        key = "INT5-10"
    elif i <50:
        key = "INT10-50"
    else:
        key = "INT50-inf"

    value = {"count":1}

    # Only pass/call the mapperoutput only if data["year_week"]=="2020-52"
    #we have the value and the key and can pass it to the mapperoutput
    if data["year_week"]=="2020-52":
        mapperoutput(key,value)


## RecordReader Function

In [35]:
# call this function for each line in the txt file
def recordreader(line):
    #in our case, each line is already a JSON object so we just need 
    # to convert from text to JSON
    data=json.loads(line)
    
    #we have now the data and can pass it to the keyvalue mapper
    keyvaluemapper(data)

##  input splitter

In [36]:
#we read line by line
for line in finput:
    #here now line is our record and we can do things with it
    if (line!=""):
        recordreader(line)
finput.close()
foutput.close()

## sort the output file. The result will be in a file called lab5mapperout_sorted.txt

In [37]:
! sort < lab5mapperout.txt > lab5mapperout_sorted.txt